In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# RNN


    http://hunkim.github.io/ml/
    
    
    CNN : 하나를 여러개로 분리해서 마지막에 다시 합치는 방식 CNN의 알고리즘
            현재 데이터를 분석하는 하나의 알고리즘일 뿐이다.
            
    RNN : 단계별로 그전에 처리됐던 결과를 또 조합시켜서 계산하고 또 이 결과를
          다른 레이어에 조합시켜서 이걸 계속 반복시켜 버림
          
          I google at work
          
          I work at google
          
          같은 google인데 위에선 동사고 밑에선 명사이다
          CNN은 이걸 구별 못한다.
          RNN은 위치에 따른 품사 구별을 할 수 있게 된다. 앞뒤 단어에 따라 
          
          그전에 뭐가 입력되었는지 알아야하기 때문에 RNN을 쓰게 되면 좋다 
          
          
    
        주로 사용하는곳
        
        음성인식
        번역
        질문, 대화
        
        
        
    강의슬라이드

        알고리즘 그림이 있음

        원랜 이걸 하나만 썼는데 여러가지를 쓰게 되는 알고리즘이다.

        그래서 그전에 어떤값이 들어왔는지를 고려할 수 있게 된 것이다. 

        상단값  맨위 - 다항분류
        state값 중간 - 이진분류, 감정분류

    
    
    실습 슬라이드
        hihello를 뽑는 실습        
        여러 단어를 분석해야하기 때문에 softmax를 사용
        
        
        Sentiment Analysis  - 감정분석
        감정을 뽑아 내서 분석
        
    
    히든 계층이 어떻게 될까
            
        
             y1                      y2
             
             ↓                      ↓   
       -------------            -------------
      |             |          |             |
      |             |          |             | 
      |      h1     |   --->   |      h2     |   ---> 
      |             |          |             |
      |      가중치w|          |      가중치w|
      |     입력값 x|          |     입력값 o|
       --------------           --------------
            ↑                        ↑
            x1                        x2
            
            
        * 초기 입력값은 없다. 
            두번째 계산 부터는 전에 계산되었던 값이 넘어와서 영향을 미치게 된다. 
            
            
    CNN 의
    
    RNN 의 Back propergation 이 어떻게 이루어지나
        값은 하난데 가능한가?
            -> 시간에 따라 BP를 한다. 
            
        
        
### RNN의 기본 알고리즘
    
    Vanila RNN 이라 한다.  - 위에서 설명한 것
    


## RNN 실습 (Vanila RNN)

        그냥 데이터를 입력했을 때 결과가 어떻게 나오는지

In [2]:
from tensorflow.contrib import rnn

In [12]:
tf.reset_default_graph()
# 입력값 준비 : hello

# 값을 숫자로 바꿔야함 

h = [1,0,0,0]  
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
    # 왜 4개?  중복된것을 빼고 네개만 한다. 
    

# cell 준비 (히든계층?)
# LSTMCell은 나중에 하고
# BasicRNNCell

cell = tf.contrib.rnn.BasicRNNCell(num_units=3)
                                # 히든 개수를 적어준다. 이걸 수정하면 출력의 갯수도 적은 수만큼 나온다.

# 입력 데이터 준비
x_data = np.array([[h,e,l,l,o]], dtype=np.float32)  
    # 시퀀스가 5개 되게 한다. 그 다음 글자가 무엇이 나올지 예측
    # 문자열이기 때문에 입력값이 최대 3차원이 되어야 한다. 
    # 그 전에 CNN할 땐 이미지여서 4차원으로 하였다. 
# print(x_data)



# RNN 설정
output, _state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    # 얘는 결과가 두개다
    # 한개는 출력
    # 한개는 옆으로 가는 state

    
# 세션 준비

sess = tf.Session()

sess.run(tf.global_variables_initializer())
    # 항상 초기화 할것
    
print(sess.run(output))
print("---------------")
print(sess.run(_state))

[[[ 0.01837398  0.4150799   0.26174298]
  [ 0.39704606 -0.29351166  0.3648948 ]
  [-0.45316276 -0.40126407  0.33250588]
  [-0.3790015  -0.73852766  0.64487135]
  [-0.10647471 -0.20027146  0.31252015]]]
---------------
[[-0.10647471 -0.20027146  0.31252015]]


    위 결과차례대로 h, e, l, l, o 값이고
    
    num_units = 3으로 적었기 때문에 3열로 값이 나온다. 
    
    마지막 output 값과
    마지막 state 값은 같아야 한다. 

In [14]:
tf.reset_default_graph()
tf.set_random_seed(777)


# I work at google

# I google at work

### one-hot encoding
# i work at google  = [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]]
# i google at work  = [[1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0]]

# 3차원으로 데이터 준비  (위 두 문장)
inputs = np.array([[[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]], 
                 [[1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0]]])


tf_inputs = tf.constant(inputs, dtype=tf.float32)

# cell준비 (기본cell)
cell = tf.contrib.rnn.BasicRNNCell(num_units=3)

# 실행
output, _state = tf.nn.dynamic_rnn(cell, tf_inputs, dtype=tf.float32)


sess = tf.Session()

sess.run(tf.global_variables_initializer())
    # 항상 초기화 할것
    
print(sess.run(output))
print("---------------")
print(sess.run(_state))
print("---------------")
# 각 W값 확인
variable_names = [v.name for v in tf.trainable_variables()]
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)

[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324395]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641277  0.70696247 -0.7512605 ]]]
---------------
[[-0.36491966  0.8857268  -0.02324395]
 [-0.11641277  0.70696247 -0.7512605 ]]
---------------
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


    우리가 한건 훈련이나 그런건 아니다.
    입력했을 때 어떤 값이 나오는지를 확인만 한것 뿐이다.
    
    맨 밑의 값 두개는 
    
    W, b의 값들이다
    W : 갯수가 들어간다. 
    b : num_units=3 이라서 3개이다.
    
    
    
    지금은 우리가 짧은 단어로 했기 때문에 잘 나오는데
    데이터가 커지면 바닐라 RNN으로 처리하기 어렵다
    
    그래서 복잡한 데이터를 하기 위해 나온 알고리즘이 LSTM이다. 
    
    LSTM (Long?)
    

## hihell 입력시  (실습 슬라이드)

    hihell 입력시 출력을 ihello를 출력할 수 있게

    즉 
    i입력 -> h 예측
    h -> e 예측
    e -> l 예측  이런식이다. 
    
    
    hihello 학습 과정
        - 중복된 문자 제외 하고 one_hot
    
    

In [33]:
tf.reset_default_graph()
tf.set_random_seed(777)

# 중복된 글자 제거하고 모아둔다.
idx2char = ['h', 'i', 'e', 'l', 'o']  

# one_hot 인코딩

x_one_hot = [[[1, 0, 0, 0, 0],       # h 0
              [0, 1, 0, 0, 0],       # i 1
              [1, 0, 0, 0, 0],       # h 0
              [0, 0, 1, 0, 0],       # e 2
              [0, 0, 0, 1, 0],       # l 3
              [0, 0, 0, 1, 0]]]      


x_data = [[0,1,0,2,3,3]]  # hihell
y_data = [[1,0,2,3,3,4]]  # ihello


# 파라미터 준비 
num_classes = 5  # 중복된거 빼고 카테고리 갯수가 5이다
input_dim = 5    # 입력의 갯수. 즉 위에랑 같은 값임.
hidden_size = 5  # 히든 레이어 사이즈, 이것도 같은 말임
batch_size = 1
sequence_len = 6
learning_rate = 0.1

# X, y 변수 준비
# 문자열은 3차원이다. 
# shape = [batchsize, sequence length, input dimension]
X = tf.placeholder(tf.float32, shape = [None, sequence_len, input_dim])
y = tf.placeholder(tf.int32, shape = [None, sequence_len])


# rnn 모델 작성
# cell준비 (기본cell)
cell = tf.contrib.rnn.BasicRNNCell(num_units=5)  
    # num_units가 결국 input_dim이랑 같다. 또는 hidden_size

initial_state = cell.zero_state(batch_size, tf.float32)
    # 셀의 크기만큼 0으로 채운다는 것, 그리고 밑에 매개변수로 넘겨준다.
    # 셀의 값이 어떤게 있는지 모르기 때문에 0으로 초기화한 것이다. 
    # 이렇게 하면 잡음을 없앨 수 있음 (??)

# 실행
outputs, _state = tf.nn.dynamic_rnn(cell, inputs=X, dtype=tf.float32, 
                                  initial_state=initial_state)


# 평면화
    # 아깐 flatten이란 함수를 썼다만
    # 지금은 그냥 reshape을 사용, 2차원으로 고침
X_for_fc = tf.reshape(outputs, [-1, hidden_size])


# FC   실제 훈련시키기
# W, b, logit

outputs = tf.contrib.layers.fully_connected(inputs = X_for_fc, num_outputs=num_classes, activation_fn=None)
    # 입력값, 2차원 변환 시킨값
    # 출력값, num_classes 
    # activation_fn 

outputs = tf.reshape(outputs, [batch_size, sequence_len, num_classes])
        # 3차원으로 다시 재변환

weights = tf.ones([batch_size, sequence_len])


sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights= weights)
    # 로짓 : 우리가 데이터를 통과시키지 않은 데이터 (3차원으로 넣어야함)
        # 그런데 우리가 한건 2차원이였음 그래서 3차원으로 재조정 해줘야한다.
    # 정답 
    # 가중치
    
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


# 맞았는지 틀렸는지
prediction = tf.argmax(outputs, 2)

with tf.Session() as sess :
    # 항상 global_variables로 초기화는 해줘야한다. 
    sess.run(tf.global_variables_initializer())
    
    for i in range(50) :
        _, cost = sess.run([train, loss], feed_dict={X:x_one_hot, y:y_data})
        
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i, "loss", cost, "     prediction : ", result, "        true Y:", y_data)
        
    result_str = [idx2char[c] for c in np.squeeze(result)]
        # squeez() 차원을 줄여주는 함수
        # result가 2차원이므로 1차원으로 줄인다. 
    print("결과 : ", ",".join(result_str))

0 loss 1.7141227      prediction :  [[3 3 3 3 3 3]]         true Y: [[1, 0, 2, 3, 3, 4]]
1 loss 1.4223528      prediction :  [[3 3 3 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
2 loss 1.2504609      prediction :  [[2 0 3 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
3 loss 1.0714287      prediction :  [[2 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
4 loss 0.9058055      prediction :  [[2 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
5 loss 0.73478097      prediction :  [[2 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
6 loss 0.58669895      prediction :  [[2 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
7 loss 0.47528145      prediction :  [[2 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
8 loss 0.3896686      prediction :  [[2 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
9 loss 0.31760624      prediction :  [[1 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
10 loss 0.2537753      prediction :  [[1 0 2 3 3 4]]         true Y: [[1, 0, 2, 3, 3, 4]]
11 loss 0.197954

    ???????????
   

# LSTM

    Long Sequence Training Memory (?)

    복잡하고 긴 문장을 초리하기 위함
    
    Gradient Vanishing를 처리하기 위함  (값을 못찾는것) 
    
    
    바닐라 알고리즘과의 차이
        Vanila : 상태선이 하나
        LSTM : 상태선이 두개
        
        
    

In [51]:
tf.reset_default_graph()
tf.set_random_seed(777)

inputs = np.array([[[1, 0]]])

tf_inputs = tf.constant(inputs, dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units=1)
outputs, _states = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs=tf_inputs)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run, states_run = sess.run([outputs, _states])
    print("output values")
    print(outputs_run)
    print("\nmemory cell value ")
    print(states_run.c)
    print("\nhidden state value ")
    print(states_run.h)

output values
[[[0.09927537]]]

memory cell value 
[[0.18134572]]

hidden state value 
[[0.09927537]]


In [52]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

df = pd.DataFrame(paragraph_dict_list)
df = df[["paragraph", "category"]]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


In [53]:
# 단어별로 추출
results = set()
df["paragraph"].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [54]:
# 분할 된 단어 확인
results

{'(and',
 '1',
 '1.5',
 '1.5x',
 '10-9',
 '14',
 '2',
 '2014',
 '33/1',
 'a',
 'about',
 'action',
 'after',
 'against',
 'ago',
 'ai',
 'alisson',
 'all',
 'almost',
 'also',
 'although',
 'alto',
 'amazing',
 'american',
 'and',
 'angelique',
 'any',
 'apps',
 'are',
 'area',
 'around',
 'as',
 'at',
 'australia',
 'avoided',
 'azarenka',
 'back',
 'battle',
 'be',
 'beef',
 'been',
 'before',
 'began',
 'began—too',
 'being',
 'better',
 'between',
 'big',
 'biscuits',
 'bit',
 'bookmark',
 'bookmarked',
 'both',
 'bottom',
 'brazilian',
 'bread',
 'breakthroughs',
 'briton',
 'brunch',
 'burger',
 'burgers',
 'business',
 'busy',
 'but',
 'by',
 'came',
 'campaigns',
 'can',
 'card',
 'caroline',
 'case',
 'certainly',
 'champion',
 'champions',
 'chance',
 'chances',
 'chicken',
 'city',
 'clay',
 'clean',
 'commented',
 'competition',
 'complain',
 'confidence',
 'contrast',
 'could',
 'course',
 'courteous',
 'courts',
 'coworkers',
 'crisp',
 'croatia',
 'crumbs',
 'culminating

In [55]:
# 쓰기 좋게 dict 형식으로 변경
idx2word = dict(enumerate(results))
idx2word

{0: 'williams',
 1: 'without',
 2: 'days',
 3: 'match',
 4: 'you',
 5: 'not',
 6: 'possible',
 7: 'contrast',
 8: 'wimbledon',
 9: 'will',
 10: 'missed',
 11: 'full',
 12: 'illness',
 13: 'those',
 14: 'bread',
 15: 'laughed',
 16: 'stunning',
 17: 'pulled',
 18: 'pull',
 19: 'ostapenko',
 20: 'succeed',
 21: 'russia',
 22: 'tournaments',
 23: 'last',
 24: 'green',
 25: 'nadal',
 26: 'off',
 27: 'leicester',
 28: 'nn7',
 29: 'names',
 30: 'about',
 31: 'more',
 32: 'champion',
 33: 'expensive',
 34: 'got',
 35: 'came',
 36: 'their',
 37: 'over',
 38: 'semi-final',
 39: 'around',
 40: 'having',
 41: 'also',
 42: 'kerber',
 43: 'chances',
 44: 'once',
 45: 'no1',
 46: 'improved',
 47: 'performances',
 48: 'djokovic-nadal',
 49: 'flair',
 50: 'us',
 51: 'difficult',
 52: 'ai',
 53: 'at',
 54: 'enjoyed',
 55: 'say',
 56: 'only',
 57: 'potential',
 58: 'caroline',
 59: 'seeded',
 60: 'when',
 61: 'form',
 62: 'nederer',
 63: 'fries',
 64: 'lead',
 65: 'returning',
 66: 'served',
 67: 'doubl

In [56]:
idx2word[396]
idx2word[100]

'fairly'

In [57]:
# 반대로 문자를 넣으면 인덱스를 알려주는 것
word2idx = {v:k for k, v in idx2word.items()}

In [58]:
word2idx["bread"]
word2idx["tournament"]

319

In [59]:
def encode_paragraph(paragraph):
    words = paragraph.split(" ")
    encoded = []
    for word in words:
        encoded.append([word2idx[word]])
    
    return encoded

def encode_category(category):
    if category == "food":
        return [1, 0]
    else:
        return [0, 1]
    
def word_cnt(paragraph):
    return len(paragraph.split(" "))

In [62]:
df["enc_paragraph"] = df.paragraph.apply(encode_paragraph)
df["enc_category"] = df.category.apply(encode_category)
df["seq_length"] = df.paragraph.apply(word_cnt)

In [63]:
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[281], [248], [337], [501], [483], [299], [32...","[1, 0]",53
1,service can be slower during busy hours but ou...,food,"[[272], [469], [138], [202], [91], [308], [269...","[1, 0]",19
2,portions are huge both french toast and their ...,food,"[[276], [201], [511], [220], [516], [235], [50...","[1, 0]",42
3,we started with apps going the chicken and waf...,food,"[[507], [113], [222], [513], [359], [151], [26...","[1, 0]",43
4,the biscuits and gravy was too salty two peopl...,food,"[[151], [231], [505], [93], [252], [382], [524...","[1, 0]",82
5,the garlic fries were a great starter (and a h...,food,"[[151], [206], [63], [167], [139], [85], [523]...","[1, 0]",24
6,our meal was excellent i had the pasta ai form...,food,"[[521], [283], [252], [372], [428], [210], [15...","[1, 0]",50
7,what i enjoy most about palo alto is so many r...,food,"[[491], [428], [484], [185], [30], [423], [506...","[1, 0]",43
8,the drinks came out fairly quickly a good two ...,food,"[[151], [304], [35], [96], [100], [128], [139]...","[1, 0]",49
9,despite the not so good burger the service was...,food,"[[333], [151], [5], [365], [107], [440], [151]...","[1, 0]",82


In [64]:
# dynamic_rnn  : 불규칙 길이를 입력값을 동적으로 처리할 수 있게 함.

# static_rnn : 같은 길이 입력값만 받음

# 데이터의 전체길이를 아는게 중요하다. 


# 문장의 최저길이를 알아내기
max_word_cnt = 0
for row in df["paragraph"]:
    if len(row.split(" ")) > max_word_cnt:
        max_word_cnt = len(row.split(" "))
        
print(max_word_cnt)

91


    한줄씩 비교를 해서 큰값을 남기고 계속 차례로 비교해서 남기면
    
    제일 큰 값이 남게 되게 하는 간단한 알고리즘이다. 

In [65]:
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
        
    return enc_paragraph

In [66]:
df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

In [67]:
print(df["enc_paragraph"][10])

[[151], [417], [188], [519], [122], [378], [79], [58], [477], [117], [42], [505], [361], [50], [348], [32], [301], [105], [123], [324], [139], [515], [503], [439], [151], [265], [185], [171], [362], [20], [161], [394], [371], [342], [105], [353], [41], [54], [151], [45], [528], [226], [151], [23], [228], [199], [394], [209], [249], [503], [36], [158], [350], [504], [297], [209], [125], [298], [151], [418], [417], [459], [501], [151], [514], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


In [68]:
# 입력값들을 배열로 변환
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())
seq_length = np.array(df.seq_length.tolist())

In [69]:
train_X = enc_paragraph
train_y = enc_category

In [70]:
print(train_X.shape)
print(train_y.shape)

(20, 91, 1)
(20, 2)


In [71]:
# 모델구축

tf.reset_default_graph()
tf.set_random_seed(777)

learning_rate = 0.001
n_epochs = 300

X = tf.placeholder(tf.float32, [None, max_word_cnt, 1])
y = tf.placeholder(tf.int32, [None, 2])

embedding = tf.layers.dense(X, 5)

cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
output, state = tf.nn.dynamic_rnn(cell, embedding, dtype=tf.float32, sequence_length=seq_length)

dense_layer = tf.layers.dense(state.h, 32)
logits = tf.layers.dense(dense_layer, 2)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [72]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        _, cost = sess.run([train, loss], feed_dict={X:train_X, y:train_y})
        if epoch % 50 == 0:
            pred = tf.nn.softmax(logits)
            correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))
            cur_acc = accuracy.eval({X:train_X, y:train_y})
            print("epoch:" + str(epoch) + ", loss:" + str(cost) + ", accuracy:" + str(cur_acc))

epoch:0, loss:0.86892205, accuracy:0.5
epoch:50, loss:0.39301464, accuracy:0.9
epoch:100, loss:0.055023037, accuracy:1.0
epoch:150, loss:0.0067080595, accuracy:1.0
epoch:200, loss:0.0021789588, accuracy:1.0
epoch:250, loss:0.001160603, accuracy:1.0
